In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from buck.analysis.basics import ingest_images

# Your existing ingestion
fpath = "..\\images\\squared\\*_NDA.png"
images, ages = ingest_images(fpath)

from buck.analysis.basics import split_data

Xtr_og, ytr_og, Xval, yval, Xte, yte_onehot, ages, l_map = split_data(images, ages)

In [ ]:
from buck.analysis.basics import homogenize_data

augment_multiplier = 40
X_train_flat, y_train_flat, X_test_flat, y_true, label_mapping, num_classes = homogenize_data(Xtr_og, ytr_og, Xte,yte_onehot, l_map, augment_multiplier)

In [4]:
# Random Forest
from buck.classifiers.random_forest import (
    _optimize_rs, _optimize_nest, _optimize_max_d, _optimize_crit, _optimize_cw, _optimize_mss, _optimize_msl, _optimize_mwfl, _optimize_mf, _optimize_mln, _optimize_mid
)

# Shorten parameters
Xtr_pca = X_train_pca
ytr_flat = y_train_flat
Xte_pca = X_test_pca

opts = {
    "n_estimators": 100,
    "criterion": "gini",
    "max_depth": None,
    "min_samples_split": 2,
    "min_samples_leaf": 1,
    "min_weight_fraction_leaf": 0.0,
    "max_features": "sqrt",
    "max_leaf_nodes": None,
    "min_impurity_decrease": 0.0,
    "bootstrap": True,
    "oob_score": False,
    "n_jobs": -1,
    "random_state": 42,
    "verbose": 0,
    "warm_start": False,
    "class_weight": None,
    "ccp_alpha": 0.0,
    "max_samples": None,
    "monotonic_cst": None,
}

# Optimize hyperparameters
ma_vec = []
f1_vec = []
for c in np.arange(10):
    opts, _, _ = _optimize_rs(Xtr_pca, ytr_flat, Xte_pca, y_true, opts)
    opts, _, _ = _optimize_nest(Xtr_pca, ytr_flat, Xte_pca, y_true, opts)
    opts, _, _ = _optimize_max_d(Xtr_pca, ytr_flat, Xte_pca, y_true, opts)
    opts, _, _ = _optimize_crit(Xtr_pca, ytr_flat, Xte_pca, y_true, opts)  # type: ignore
    opts, _, _ = _optimize_cw(Xtr_pca, ytr_flat, Xte_pca, y_true, opts)
    opts, _, _ = _optimize_mss(Xtr_pca, ytr_flat, Xte_pca, y_true, opts)
    opts, _, _ = _optimize_msl(Xtr_pca, ytr_flat, Xte_pca, y_true, opts)
    opts, _, _ = _optimize_mwfl(Xtr_pca, ytr_flat, Xte_pca, y_true, opts)
    opts, _, _ = _optimize_mf(Xtr_pca, ytr_flat, Xte_pca, y_true, opts)
    opts, _, _ = _optimize_mln(Xtr_pca, ytr_flat, Xte_pca, y_true, opts)
    opts, ma, f1 = _optimize_mid(Xtr_pca, ytr_flat, Xte_pca, y_true, opts)
    ma_vec.append(ma)
    f1_vec.append(f1)
    print(ma, f1)

#Best: 0.7894736842105263
#{'n_estimators': np.int64(127),
# 'criterion': 'gini',
# 'max_depth': None,
# 'min_samples_split': np.int64(2),
# 'min_samples_leaf': np.int64(1),
# 'min_weight_fraction_leaf': np.float64(0.0),
# 'max_features': 'log2',
# 'max_leaf_nodes': None,
# 'min_impurity_decrease': np.float64(0.0),
# 'bootstrap': True,
# 'oob_score': False,
# 'n_jobs': -1,
# 'random_state': np.int64(405),
# 'verbose': 0,
# 'warm_start': False,
# 'class_weight': None,
# 'ccp_alpha': 0.0,
# 'max_samples': None,
# 'monotonic_cst': None}

0.6 0.5840350877192982
0.6 0.5840350877192982
0.6 0.5840350877192982
0.6 0.5840350877192982
0.6 0.5840350877192982
0.6 0.5840350877192982
0.6 0.5840350877192982
0.6 0.5840350877192982
0.6 0.5840350877192982
0.6 0.5840350877192982


In [5]:
from buck.classifiers.autotune import optimize_all

optimize_all(Xtr_pca, ytr_flat, Xte_pca, y_true, cycles=1)

Optimizing
...bagging classifier


ValueError: max_features must be <= n_features